# **Configurando as bibliotecas do projeto**

## *Instalação de bibliotecas*

In [85]:
# Instala a biblioteca "TA-Lib" que será responsável por fornecer a implementação de alguns indicadores técnicos.
!pip install Ta-Lib

Defaulting to user installation because normal site-packages is not writeable


In [86]:
# Instala a biblioteca "tensorflow" que será responsável por fornecer a implementação dos métodos necessários para a construção da LSTM.
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


## *Importação de bibliotecas*

In [87]:
'''
    Essa célula será usada para a importação de bibliotecas.
'''

# Importa a biblioteca que será utilizada para a obtenção dos dados financeiros.
import yfinance as yf
# Importa a biblioteca que será utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa o módulo da biblioteca "datetime" que será utilizada para lidar com objetos do tipo "datetime".
from datetime import datetime
# Importa o módulo da biblioteca "typing" que será utilizado para tipar parâmetros opcionais de funções.
from typing import Optional
# Importa a biblioteca que será utilizada para a manipulação de arrays e também para o uso de algumas funções matemáticas.
import numpy as np
# Importa a biblioteca que será utilizada para calcular algumas métricas de análise técnica.
import talib
# Importa o módulo da biblioteca "sklearn" que será utilizado para escalar as features dos DataFrames que serão usados no modelo LSTM.
from sklearn.preprocessing import MinMaxScaler
# Importa um dos módulos da biblioteca "tensorflow" que será utilizado para criar o modelo LSTM. 
from tensorflow.keras.models import Sequential
# Importa um dos módulos da biblioteca "tensorflow" que será utilizado para construir as camadas que farão parte da arquitetura do modelo LSTM 
# a ser usado. 
from tensorflow.keras.layers import LSTM, Dense, Dropout
# Importa a biblioteca que será utilizada para criar gráficos interativos.
import plotly.graph_objects as go
# Importa a função "mean_squared_error" da biblioteca sklearn.metrics.
from sklearn.metrics import mean_squared_error

# **Data Engineering**

### *Configurações iniciais*

In [88]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_tickets = sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_tickets = sp500_tickets.tolist()

# Exibe a lista de tickets criada acima.
sp500_tickets

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG'

In [89]:
'''
    Essa célula será usada para definir o intervalo de tempo dos dados que serão coletados, especificando a data de início e a data de fim.
'''

# Define a data inicial cujos dados serão coletados.
start_date = datetime(2019,1,2).date()
# Define a data final cujos dados serão coletados.
end_date = datetime(2023,12,30).date()

In [90]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais para a coleta de dados, 
    incluindo a lista de tickers das companhias do S&P 500 e o intervalo de tempo.
'''

# Cria um dicionário para guardar alguns parâmetros importantes para a coleta de dados.
setup = {
    # Tickers das ações que serão usadas no código.
    "tickers": sp500_tickets,
    # Data inicial cujo alguns dados das ações de cada ticker serão coletados.
    "start_date": start_date,
    # Data final cujo alguns dados das ações de cada ticker serão coletados.
    "end_date": end_date,
    # Número de tickers com maior "V Value" que terão seus preços estimados por um modelo LSTM.
    "tickers_to_select": 5,
    # Data a partir do qual os valores dos tickers com maior "V Value" serão estimados.
    "test_initial_day": np.datetime64('2023-12-01'),
    # Tamanho das sequências temporais de dados que o modelo LSTM usará para se treinar.
    "sequence_length": 7
}

### *Obtendo dados dos tickers via yfinance (Open, High, Low, Close, Adj Close, Volume)*

In [91]:
'''
    Essa célula será usada para a obtenção de alguns dados (Open, High, Low, Close, Adj Close, Volume) 
    dos tickers presentes na variável de configuração setup.
'''

# Cria um array que guardará os DataFrames que contém os dados referentes a cada um dos tickers presentes na variável de configuração setup.
data_array = np.empty(len(setup['tickers']), dtype=object)

# Itera sobre cada um dos tickers presentes na variável de configuração setup.
for i, ticker in enumerate(setup['tickers']):
    # Faz o download dos dados do ticker em questão
    ticker_data = yf.download(ticker, setup['start_date'], setup['end_date'])
    # Adiciona um nome ao DataFrame do ticker em questão.
    ticker_data.name=ticker
    # Salva o DataFrame do ticker em questão na variável "data_array".
    data_array[i] = ticker_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$BF.B: possibly delisted; No price data found  (1d 2019-01-02 -> 2023-12-30)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

### *Filtrando os dados obtidos*

In [92]:
'''
    Essa célula será responsável por filtrar a variável "data_array" de modo que esta só contenha DataFrames não vazios.
'''

# Cria um filtro para identificar quais DataFrames presentes em "data_array" são vazios, isto é, não possuem nenhum índice.
void_filter = np.array([df.index.empty for df in data_array])

# Aplica a negação do filtro gerado acima (ou seja, filtra os DataFrames que não são vazios) no array "data_array" e armazena o resultado 
# na variável "filtered_data_array". 
filtered_data_array = data_array[~void_filter]

# Exibe uma mensagem que informa quantos tickers foram removidos na filtragem realizada acima.
print(f"Foram removidos {len(data_array) - len(filtered_data_array)} tickers que estavam vazios.")

Foram removidos 5 tickers que estavam vazios.


In [93]:
'''
    Essa célula será responsável por filtrar a variável "data_array" de modo que esta só contenha DataFrames cujos índices se iniciem em 
    setup['start_date'].
'''

# Armazena o tamanho atual do array "filtered_data_array" para que se tenha ideia ao final dessa célula, de quantos tickers 
# foram removidos com o filtro que será aplicado aqui.
filtered_data_array_initial_len = len(filtered_data_array)

# Cria um filtro para saber quais dos DataFrames presentes em "filtered_data_array" possuem datas iniciais (isto é, índices de posição 0) 
# iguais à "setup['start_date']".
start_date_filter = [df.index[0].date() == setup['start_date'] for df in filtered_data_array]

# Aplica o filtro criado acima, removendo assim todos os DataFrames presentes em "filtered_data_array" cujos índices iniciais são diferentes de 
# "setup['start_date']".
filtered_data_array = filtered_data_array[start_date_filter]

# Exibe uma mensagem informando quantos DataFrames não satisfaziam a condição do filtro criado acima, e informando também quantos DataFrames
# ainda existem em "data_array".
print(f"Foram removidos {filtered_data_array_initial_len - len(filtered_data_array)} tickers que não possuiam data inicial igual à {setup['start_date']}. Agora, todos os demais {len(filtered_data_array)} tickers \npossuem data de inicio em {setup['start_date']}.")

Foram removidos 13 tickers que não possuiam data inicial igual à 2019-01-02. Agora, todos os demais 485 tickers 
possuem data de inicio em 2019-01-02.


In [94]:
'''
    Essa célula será utilizada para mostrar que, nos dados baixados, não existem DataFrames em "data_array" cujos índices terminam em uma 
    data anterior à última data de negociação possível antes de setup['end_date'], isto é não existem DataFrames em "data_array" cujos índices
    terminam em uma data anterior à 2023-12-29.
'''

# Obtem qual é o menor dentre os últimos índices de cada um dos DataFrames presentes em "filtered_data_array"
lowest_last_date = np.min(np.array([df.index[len(df.index)-1] for df in filtered_data_array]))

# Exibe uma mensagem que informa qual é a menor data final entre todos os DataFrames presentes em "filtered_data_array" e gera uma
# conclusão a partir de tal data.
print(f"Dentre os tickers que passaram nos filtros acima, a menor data final é {lowest_last_date.date()}, que é a última data possível de negociação dentro do \nperiodo estabelecido na variável 'setup' ({setup['end_date']}). Ou seja, nenhum filtro precisa ser aplicado em relação a data final dos tickers.")

Dentre os tickers que passaram nos filtros acima, a menor data final é 2023-12-29, que é a última data possível de negociação dentro do 
periodo estabelecido na variável 'setup' (2023-12-30). Ou seja, nenhum filtro precisa ser aplicado em relação a data final dos tickers.


In [95]:
'''
    Essa célula verifica se algum dos DataFrames presentes em "filtered_data_array" possui valores nulos.
'''

# Cria uma lista auxiliar que conterá valores booleanos em todas as suas posições. Sendo que, para qualquer posição "i" dessa lista, 
# se tal posição contiver o valor True, indica que o DataFrame que ocupa essa mesma posição "i" na variável "filtered_data_array" 
# possui valores nulos. Caso contrário, indica que o DataFrame que ocupa tal posição em "filtered_data_array" não possui valores nulos.
aux_list = [df.isnull().values.any() for df in filtered_data_array]

# Exibe mensagens indicando se existem ou não DataFrames com valores nulos na variável "filtered_data_array".
if True in aux_list: 
    print("Existem DataFrames em 'filtered_data_array' que possuem valores nulos.")
else: 
    print("Não existem DataFrames em 'filtered_data_array' que possuam valores nulos.")

Não existem DataFrames em 'filtered_data_array' que possuam valores nulos.


In [96]:
'''
    Essa célula verifica se todos os DataFrames presentes em "filtered_data_array" possuem os mesmos índices.
'''

# Vamos tomar como referência os índices do DataFrame que ocupa a posição 0 da variável "filtered_data_array".
reference_indexes = filtered_data_array[0].index

# Verifica se todos os DataFrames presentes em "filtered_data_array" possuem os mesmos índices.
areAllIndexesEquals = all(df.index.equals(reference_indexes) for df in filtered_data_array)

if areAllIndexesEquals:
    print("Todos os DataFrames presentes em 'filtered_data_array' possuem os mesmos índices.")
else:
    print("Existem DataFrames presentes em 'filtered_data_array' que possuem índices distintos.")

Todos os DataFrames presentes em 'filtered_data_array' possuem os mesmos índices.


# **Feature Engineering**

#### *Gerando os retornos aritméticos diários dos tickers*

In [97]:
def get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Cria uma série temporal contendo os retornos diários do ticker em questão.
    ticker_daily_arithmetic_returns = daily_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno diário do "dia 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_daily_arithmetic_returns = ticker_daily_arithmetic_returns[1:]
    
    # Retorna a série temporal resultante.
    return ticker_daily_arithmetic_returns

#### *Gerando os retornos logarítmicos diários dos tickers*

In [98]:
def get_ticker_daily_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos logarítmicos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos diários do ticker em questão
    ticker_daily_arithmetic_returns = get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos diários do ticker em questão em sua versão logarítmica.
    ticker_daily_logarithmic_returns = ticker_daily_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna a série temporal resultante.
    return ticker_daily_logarithmic_returns

#### *Gerando uma métrica de volatilidade móvel*

In [99]:
def get_ticker_moving_volatility(daily_adjusted_closing_prices: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Essa função calcula a volatilidade móvel de "time_interval" dias de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
            time_interval (int): Quantidade de dias da janela móvel.
        Return:
            moving_volatility (pd.Series): Série temporal que representa a evolução da volatilidade móvel de "time_interval" dias do ticker
                                           em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso o parâmetro 'time_interval' não seja do tipo int, um erro é retornado.
    if not isinstance(time_interval, int):
        raise TypeError("É esperado que o parâmetro 'time_interval' seja do tipo int.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a volatilidade móvel de "time_interval" dias da série temporal "daily_adjusted_closing_prices".
    moving_volatility = daily_adjusted_closing_prices.rolling(window=time_interval).std()
    
    # Remove os primeiros (time_interval - 1) dados da série temporal gerada acima, isto é, remove os dados NaN de tal série temporal.
    moving_volatility = moving_volatility[time_interval-1:]

    # Retorna o resultado obtido.
    return moving_volatility

#### *Gerando uma métrica de volume de negociação móvel*

In [100]:
def get_ticker_moving_volume(daily_volumes: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Calcula o volume de negociação móvel de "time_interval" dias para uma série temporal de preços de um certo ticker.  
        Args:
            daily_volumes (pd.Series): Série temporal com os volumes de negociação diários do ticker em questão.
            time_interval (int): Número de dias para a janela móvel.
        Return:
            moving_volume (pd.Series): Série temporal do volume de negociação móvel calculado para o intervalo de dias especificado. 
        Errors:
            TypeError: É esperado que o parâmetro "daily_volumes" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_volumes" contenha apenas números.
    '''
    
    # Verifica se o parâmetro 'daily_volumes' é do tipo pd.Series.
    if not isinstance(daily_volumes, pd.Series):
        raise TypeError("O parâmetro 'daily_volumes' deve ser do tipo pd.Series.")
    
    # Verifica se o parâmetro 'time_interval' é do tipo int.
    if not isinstance(time_interval, int):
        raise TypeError("O parâmetro 'time_interval' deve ser do tipo int.")
    
    # Verifica se a série temporal contém apenas dados numéricos.
    if not pd.api.types.is_numeric_dtype(daily_volumes):
        raise ValueError("A série temporal 'daily_volumes' deve conter apenas números.")
    
    # Calcula o volume de negociação móvel usando a janela de 'time_interval' dias.
    moving_volume = daily_volumes.rolling(window=time_interval).mean()
    
    # Remove os valores NaN resultantes do cálculo acima.
    moving_volume = moving_volume.dropna()

    # Retorna o resultado obtido.
    return moving_volume

#### *Gerando algumas métricas de análise técnica (EMA , RSI , ATR , MOM) e adicionando novas features aos dados*

In [101]:
# Cria uma cópia do array "filtered_data_array" para inserir novas features sem eventual prejuizo as já existentes.
dfs = filtered_data_array

In [102]:
# Itero por cada DataFrame presente na variável "dfs" para adicionar novas features a cada um deles.
for df in dfs:
    
    # Calcula os retornos logarítmicos diários com base nos preços ajustados de fechamento e adiciona uma nova coluna 'Log Returns' ao DataFrame.
    df['Log Returns'] = get_ticker_daily_logarithmic_returns(df['Adj Close'])
    
    # Calcula a volatilidade móvel de 14 dias com base nos preços ajustados de fechamento e adiciona uma nova coluna 'MOV_VOL_14' ao DataFrame.
    df['MOV_VOLATILITY_14'] = get_ticker_moving_volatility(df['Adj Close'], 14)
    
    # Calcula o volume de negociação móvel de 14 dias com base nos preços ajustados de fechamento e adiciona uma nova coluna 'MOV_VOLUME_14' 
    # ao DataFrame.
    df['MOV_VOLUME_14'] = get_ticker_moving_volume(df['Volume'], 14)
    
    # Calcula a Média Móvel Exponencial (EMA) de 14 dias. A EMA é usada no lugar da Média Móvel Simples (SMA) para reduzir o atraso de sinal 
    # típico das médias móveis.
    # [Indicador de média móvel]
    df['EMA_14'] = talib.EMA(df['Adj Close'], timeperiod=14)
    
    # Calcula o Índice de Força Relativa (RSI) de 14 dias, que é um indicador de momentum utilizado para medir a velocidade e a mudança de
    # movimentos de preços.
    # [Indicador de momentum]
    df['RSI_14'] = talib.RSI(df['Adj Close'], timeperiod=14)
    
    # Calcula o Average True Range (ATR) de 14 dias, que é um indicador de volatilidade utilizado para medir a volatilidade do ativo com 
    # base na amplitude de preços (máxima, mínima e fechamento).
    # [Indicador de volatilidade]
    df['ATR_14'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    # Calcula o Momentum (MOM) de 14 dias, que é um indicador de momentum usado para identificar a força da tendência de um ativo.
    # [Indicador de momentum]
    df['MOM_14'] = talib.MOM(df['Adj Close'], timeperiod=14)
    
    # Adiciona uma coluna 'Ticker' ao DataFrame para identificar o ticker associado a cada conjunto de dados.
    df['Ticker'] = [df.name for _ in range(len(df))]
    
# Calcula um índice aleatório entre 0 e len(dfs).
random_index = np.random.randint(0,len(dfs))

# Exibe o resultado obtido com a operação dessa célula para o DataFrame na posição "random_index" da variável "dfs".
dfs[random_index]

# Observação: valores NaN foram inseridos nos DataFrames juntamente com as novas features. Esses valores NaN são inerentes ao cálculo de alguns
# indicadores técnicos, uma vez que eles requerem um período de ajuste.

,Open,High,Low,Close,Adj Close,Volume,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,,,,,,
2019-01-02,146.649994,148.000000,145.000000,145.899994,137.548737,260000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POOL
2019-01-03,146.990005,146.990005,141.820007,144.369995,136.106354,215200,-0.010542,NaN,NaN,NaN,NaN,NaN,NaN,POOL
2019-01-04,145.820007,149.889999,145.089996,149.279999,140.735291,162000,0.033444,NaN,NaN,NaN,NaN,NaN,NaN,POOL
2019-01-07,149.679993,151.429993,148.600006,150.149994,141.555496,139500,0.005811,NaN,NaN,NaN,NaN,NaN,NaN,POOL
2019-01-08,151.009995,151.059998,149.020004,149.830002,141.253799,163800,-0.002134,NaN,NaN,NaN,NaN,NaN,NaN,POOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,395.829987,397.760010,393.100006,396.109985,392.405090,178000,0.001212,18.510585,396964.285714,375.631987,70.963958,10.068693,42.072723,POOL
2023-12-26,396.660004,402.209991,395.500000,400.890015,397.140411,199800,0.011995,18.492598,389928.571429,378.499777,72.859206,9.828786,46.847687,POOL
2023-12-27,402.359985,405.010010,399.480011,403.739990,399.963715,193400,0.007084,18.929313,378607.142857,381.361635,73.950932,9.521730,40.844299,POOL


#### *Eliminando redundâncias nas features e lidando com os valores nulos*

*Como o indicador ATR_14 é calculado a partir das variáveis "High" e "Low", essas duas variáveis se tornam redundantes para o treinamento do modelo. Além disso, utilizaremos a coluna "Adj Close" como variável alvo. Como "Adj Close" é uma versão ajustada da variável "Close", esta última também se torna redundante e pouco relevante como feature. Da mesma forma, sem a variável "Close", a variável "Open" perde parte de sua relevância. Ademais, temos duas features que medem aspectos semelhantes de maneiras diferentes: "Volume" e "MOV_VOLUME_14". Como a feature "MOV_VOLUME_14" parece ser mais relevante para o escopo deste projeto, a manutenção da feature "Volume" não é necessária. Portanto, remover essas features do conjunto de dados é uma boa abordagem.*

In [103]:
# Itera sobre cada um dos DataFrames presentas na variável "dfs" para remover as features citadas no texto da célula acima.
for df in dfs:
    # Remove as colunas 'Open', 'High', 'Low' e 'Close' de cada DataFrame presente em "dfs".
    df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

# Calcula um índice aleatório entre 0 e len(dfs).
random_index = np.random.randint(0,len(dfs))

# Exibe o resultado obtido com a operação dessa célula para o DataFrame na posição "random_index" da variável "dfs".
dfs[random_index]

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,
2019-01-02,59.009998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AKAM
2019-01-03,57.700001,-0.022450,NaN,NaN,NaN,NaN,NaN,NaN,AKAM
2019-01-04,59.380001,0.028700,NaN,NaN,NaN,NaN,NaN,NaN,AKAM
2019-01-07,61.240002,0.030843,NaN,NaN,NaN,NaN,NaN,NaN,AKAM
2019-01-08,62.450001,0.019566,NaN,NaN,NaN,NaN,NaN,NaN,AKAM
...,...,...,...,...,...,...,...,...,...
2023-12-22,119.620003,0.001087,2.311506,1.413850e+06,117.774475,74.279862,1.389361,3.720001,AKAM
2023-12-26,119.480003,-0.001171,2.318194,1.398900e+06,118.001879,73.019474,1.326550,3.860001,AKAM
2023-12-27,119.000000,-0.004026,2.152567,1.418086e+06,118.134961,68.714373,1.324653,4.529999,AKAM


*Observe que há dados faltantes nos DataFrames, principalmente devido ao período de ajuste dos indicadores técnicos. Como esses indicadores são calculados com uma janela de 14 dias, os primeiros 14 dias podem conter valores nulos. Para resolver isso, podemos remover as linhas com valores faltantes. Como estamos lidando com no máximo 14 linhas faltantes em um total de 1.258 linhas por DataFrame, essa remoção representa apenas uma pequena fração da amostra. Portanto, a quantidade de dados disponível para o treinamento do modelo não será significativamente afetada.*

In [104]:
# Itera sobre cada um dos DataFrames presentas na variável "dfs" para remover as linhas com valores NaN presentes em cada um deles.
for df in dfs:
    # Remove todas as linhas com valores NaN de cada DataFrame na lista "dfs".
    df.dropna(inplace=True)

# Calcula um índice aleatório entre 0 e len(dfs).
random_index = np.random.randint(0,len(dfs))

# Exibe o resultado obtido com a operação dessa célula para o DataFrame na posição "random_index" da variável "dfs".
dfs[random_index]

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,
2019-01-23,14.790000,-0.050752,0.637164,8.973643e+05,16.146333,46.331522,0.957000,-0.540000,MRNA
2019-01-24,14.640000,-0.010194,0.752879,8.162214e+05,15.945489,45.336472,0.928286,-0.860000,MRNA
2019-01-25,14.910000,0.018275,0.802907,7.655357e+05,15.807424,47.521259,0.899123,-2.049999,MRNA
2019-01-28,14.450000,-0.031338,0.914363,6.991857e+05,15.626434,44.274537,0.867757,-1.820001,MRNA
2019-01-29,14.630000,0.012380,0.934768,6.063357e+05,15.493576,45.834031,0.836131,-2.320001,MRNA
...,...,...,...,...,...,...,...,...,...
2023-12-22,94.889999,0.040102,5.210992,6.627893e+06,85.438824,69.662149,4.704113,15.019997,MRNA
2023-12-26,97.330002,0.025389,6.018648,6.748800e+06,87.024314,71.952821,4.612962,19.050003,MRNA
2023-12-27,100.730003,0.034336,7.094485,6.820829e+06,88.851739,74.807297,4.544179,20.130005,MRNA


####

# **Filtrando as ações**

Para esse projeto, buscaremos trabalhar com ações que possuam, para um período de 14 dias, alto volume de negociação e baixa volatilidade. Para tal, filtraremos todas as ações obtidas através da métrica $V$ abaixo:

$$V = \dfrac{V_{l}}{V_{t}}$$

Onde,

$V_{l} =$ Volume móvel dos últimos 14 dias.

$V_{t} =$ Volatilidade móvel dos últimos 14 dias.

In [105]:
# Cria um DataFrame vazio para armazenar o "V Value" de cada DataFrame presente em "dfs", juntamente do nome do ticker que cada um desses
# DataFrames representam.
v_value_results = pd.DataFrame()  

# Itera sobre todos os DataFrames presentes na variável "dfs"  para criar o DataFrame "aux_df" que será utilizado para descobrir quais tickers,
# dentre os tickers presentes na variável de configuração "setup" possuem o maior "V Value".
for df in dfs:  
    # Obtém o último valor da feature 'MOV_VOLUME_14' do DataFrame em questão.
    df_mov_volume = df.tail(1)['MOV_VOLUME_14']  
    
    # Obtém o último valor da feature 'MOV_VOLATILITY_14' do DataFrame em questão.
    df_mov_volatility = df.tail(1)['MOV_VOLATILITY_14']  
    
    # Obtem o nome do ticker que representa os dados do DataFrame em questão.
    df_ticker = df.tail(1)['Ticker'] 
    
    # Calcula o 'V_Value' do DataFrame em questão.
    df_v_value = df_mov_volume / df_mov_volatility  
    
    # Cria um DataFrame auxiliar com os valores calculados.
    aux_df = pd.DataFrame({  
        "V_Value": df_v_value,
        "Ticker": df_ticker
    })
    
    # Concatena o DataFrame auxiliar com o DataFrame de resultados, acumulando os dados e gerando o resultado final ao final do loop.
    v_value_results = pd.concat([v_value_results, aux_df])  

In [106]:
# Ordena o DataFrame 'v_value_results' com base na coluna 'V_Value' em ordem crescente.
v_value_results = v_value_results.sort_values(by="V_Value")  

# Exibe o DataFrame ordenado.
v_value_results

,V_Value,Ticker
Date,,
2023-12-29,1.393141e+02,NVR
2023-12-29,3.201976e+03,AZO
2023-12-29,3.805557e+03,MTD
2023-12-29,4.296227e+03,BKNG
2023-12-29,1.464196e+04,FICO
...,...,...
2023-12-29,1.037063e+08,AMCR
2023-12-29,1.110892e+08,AAL
2023-12-29,1.149379e+08,F


In [107]:
# Obtem o nome dos tickers com maior "V Value" e os junta em um ndarray.
selected_tickers = v_value_results.tail(setup['tickers_to_select'])['Ticker'].to_numpy() 
# Exibe o array que contém os tickers que possuem o maior "V Value".
selected_tickers

array(['AMCR', 'AAL', 'F', 'T', 'NVDA'], dtype=object)

In [108]:
# Cria uma lista vazia para armazenar o resultado final (DataFrames que passaram pelo filtro, ou seja, os que possuem o maior "V Value").
dfs_list = []

# Itera sobre cada um dos DataFrames da variável "dfs".
for df in dfs: 
    # Verifica se o nome do DataFrame está na lista de tickers selecionados.
    if df.name in selected_tickers:
        # Guarda o DataFrame que contém os dados do ticker em questão em "dfs_list".
        dfs_list.append(df)
     
# Exibe o resultado obtido (isto é, uma lista que contém os DataFrames que possuem o maior "V Value").
dfs_list

[            Adj Close  Log Returns  MOV_VOLATILITY_14  MOV_VOLUME_14  \
 Date                                                                   
 2019-01-23   7.437139     0.014553           0.086415   1.913571e+04   
 2019-01-24   7.437139     0.000000           0.088650   1.042143e+04   
 2019-01-25   7.437139     0.000000           0.085239   1.041429e+04   
 2019-01-28   7.437139     0.000000           0.091333   7.850000e+03   
 2019-01-29   7.437139     0.000000           0.093392   7.121429e+03   
 ...               ...          ...                ...            ...   
 2023-12-22   9.324051    -0.003094           0.117780   9.159857e+06   
 2023-12-26   9.324051     0.000000           0.097765   8.763057e+06   
 2023-12-27   9.362580     0.004124           0.077678   8.754850e+06   
 2023-12-28   9.352948    -0.001029           0.078750   8.373714e+06   
 2023-12-29   9.285523    -0.007235           0.078889   8.181264e+06   
 
               EMA_14     RSI_14    ATR_14    MOM

# **Data Preparation**

#### *Preparando os DataFrames que serão usados como exemplos*

In [109]:
# Escolhe aleatoriamente um dos tickers que passaram pelo filtro e o exibe.
dfs_list[np.random.randint(0, len(dfs_list))]

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,
2019-01-23,7.437139,0.014553,0.086415,1.913571e+04,7.272271,72.413860,0.062857,0.299328,AMCR
2019-01-24,7.437139,0.000000,0.088650,1.042143e+04,7.294254,72.413860,0.058367,0.299328,AMCR
2019-01-25,7.437139,0.000000,0.085239,1.041429e+04,7.313305,72.413860,0.054198,0.299328,AMCR
2019-01-28,7.437139,0.000000,0.091333,7.850000e+03,7.329816,72.413860,0.050327,0.145827,AMCR
2019-01-29,7.437139,0.000000,0.093392,7.121429e+03,7.344126,72.413860,0.046732,0.191877,AMCR
...,...,...,...,...,...,...,...,...,...
2023-12-22,9.324051,-0.003094,0.117780,9.159857e+06,9.250234,58.965231,0.174854,0.192645,AMCR
2023-12-26,9.324051,0.000000,0.097765,8.763057e+06,9.260076,58.965231,0.170222,0.298599,AMCR
2023-12-27,9.362580,0.004124,0.077678,8.754850e+06,9.273743,60.395799,0.167349,0.298601,AMCR


In [110]:
# Itera sobre cada um dos DataFrames presentes na variável "dfs" para adicionar um nome a eles e também para remover a feature "Ticker" de cada um.
for df in dfs_list:
    # Seta o nome do DataFrame em questão como sendo o nome do ticker que este DataFrame representa.
    df.name = df['Ticker'].values[0]
    # Remove a coluna "Ticker" do DataFrame em questão (Dado o modo que treinaremos a LSTM, tal coluna não é necessária).
    df.drop(['Ticker'], axis=1, inplace=True)
    
# Escolhe aleatoriamente um dos tickers que passaram pelo filtro e o exibe, no intuito de mostrar as modificações feitas nessa célula.
dfs_list[np.random.randint(0, len(dfs_list))]

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14
Date,,,,,,,,
2019-01-23,15.227885,0.010086,0.265821,4.542227e+07,14.960152,72.704962,0.455330,0.979866
2019-01-24,15.084925,-0.009432,0.166245,4.417300e+07,14.976788,67.863035,0.456255,0.817614
2019-01-25,15.114506,0.001959,0.116466,4.307654e+07,14.995151,68.332976,0.447403,0.480624
2019-01-28,15.119437,0.000326,0.104138,4.212682e+07,15.011722,68.415895,0.428933,0.220275
2019-01-29,15.134224,0.000978,0.104542,4.097073e+07,15.028056,68.680708,0.411782,0.046950
...,...,...,...,...,...,...,...,...
2023-12-22,15.785001,0.003632,0.254131,4.357361e+07,15.764780,54.667469,0.280229,-0.104915
2023-12-26,15.804076,0.001208,0.204151,4.004928e+07,15.770019,55.120849,0.272355,-0.619954
2023-12-27,15.813613,0.000603,0.184327,3.875365e+07,15.775832,55.361216,0.265044,-0.343360


In [111]:
# Lembre que agora cada um dos DataFrames em "data_list" possuem o atributo name como sendo a sigla do ticker que tais DataFrames representam.
dfs_list[np.random.randint(0, len(dfs_list))].name

'AMCR'

#### *Dividindo os dados em treino e teste*

In [112]:
# Lembre que possuímos a variável "selected_tickers" que contém o nome dos tickers que passaram pelo filtro.

# Exibe o nome dos tickers que passaram pelo filtro.
selected_tickers

array(['AMCR', 'AAL', 'F', 'T', 'NVDA'], dtype=object)

In [113]:
def split_dfs(dfs_list: list, target: str) -> dict:
    '''
    Description:
        Esta função divide cada DataFrame de uma lista de DataFrames em dados de treino e teste e salva tais dados em um dicionário. Tal divisão
        é feita tendo como base os valores definidos nas variáveis "setup['start_date']", setup['end_date'] e "setup['test_initial_day']". Visto
        que, tal divisão deixa (aproximadamente, devido aos dias sem negociação), para cada DataFrame (df) presente em "dfs_list", 
        (setup['test_initial_day'] - setup['start_date']) amostras de treino e (setup['end_date'] - setup['test_initial_day']) 
        amostras de teste.
    Args:
        dfs_list (list): Lista de DataFrames.
        target (str): Nome da feature target.
    Return:
        train_test_data (dict): Um dicionário contendo os dados de treino e teste para cada ticker que possui um DataFrame presente em "dfs_list".
    Errors:
        TypeError: É esperado que o parâmetro "dfs_list" seja uma lista de DataFrames.
        KeyError: É esperado que todos os DataFrames presentes em "dfs_list" possuam uma coluna com o nome da variável "target".
        ValueError: É esperado que os índices de cada um dos DataFrames presentes em "dfs_list" seja uma série temporal.
    '''
    
    # Verifica se 'dfs_list' é do tipo list.
    if not isinstance(dfs_list, list):
        raise TypeError("É esperado que o parâmetro 'dfs_list' seja uma lista.")

    # Inicializa um dicionário para armazenar os dados de treino e teste para cada ticker.
    subdict = {
        'X_train': [],
        'X_train_scaled': [],
        'X_train_scaled_sequences': [],
        'X_test': [],
        'X_test_scaled': [],
        'X_test_scaled_sequences': [],
        'y_train': [],
        'y_train_scaled': [],
        'y_train_scaled_sequences': [],
        'y_test': [],
        'y_test_scaled': [],
        'y_test_scaled_sequences': [],
        'predicted': [],
        'RMSE': 0
    } # Existem formas mais eficientes e "bonitas" de se fazer algo que faça um papel semelhante ao dessa estrutura. Contudo, por questões de
    # didática e legibilidade essa estrutura será mantida assim.

    # Inicializa um dicionário onde as chaves são os nomes de cada ticker que possui um DataFrame em "dfs_list".
    train_test_data = dict.fromkeys([df.name for df in dfs_list])

    # Itera sobre cada DataFrame presente na variável "dfs_list".
    for df in dfs_list:
        
        # Verifica se o DataFrame em questão possui a coluna target.
        if target not in df.columns:
            raise KeyError(f"Todos os DataFrames devem a coluna {target}.")
        
        # Separa as features (X) e o target (y).
        X = df.drop(columns=['Adj Close'])
        y = df['Adj Close']
        
        # Verifica se o índice do DataFrame é uma série temporal.
        if not pd.api.types.is_datetime64_any_dtype(df.index):
            raise ValueError("É esperado que os índices de todos os DataFrames sejam uma série temporal.")
        
        # Separa o conjunto de séries temporais das features em dados de treino e dados de teste.
        X_train, X_test = X[X.index < setup['test_initial_day']], X[X.index >= setup['test_initial_day']]
        # Separa a série temporal do target em dados de treino e dados de teste.
        y_train, y_test = y[y.index < setup['test_initial_day']], y[y.index >= setup['test_initial_day']]
        
        # Inicializa o dicionário para o ticker atual.
        train_test_data[df.name] = subdict.copy()
        
        # Atribui os conjuntos de treino e teste ao dicionário do ticker atual.
        train_test_data[df.name]['X_train'] = X_train
        train_test_data[df.name]['X_test'] = X_test
        train_test_data[df.name]['y_train'] = y_train
        train_test_data[df.name]['y_test'] = y_test

    return train_test_data

In [114]:
# Seta o nome do target.
target = 'Adj Close'
# Separa os dados de cada DataFrame presente em "dfs_list" em dados de treino e dados de teste.
train_test_data = split_dfs(dfs_list, target)
# Obtem, de forma aleatória, o ticker que representa um dos DataFrames presentes em "dfs_list".
random_ticker = list(train_test_data.keys())[np.random.randint(0,len(train_test_data.keys()))]
# Exibe qual ticker foi escolhido aleatoriamente.
print(random_ticker)

AAL


In [115]:
# Exibe, para o ticker selecionado aleatoriamente, o conjunto de séries temporais das features que será utilizado para treino.
train_test_data[random_ticker]['X_train']

,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14
Date,,,,,,,
2019-01-23,-0.033860,0.973723,9.816450e+06,31.819527,46.839310,1.759286,-0.816790
2019-01-24,0.061572,0.774769,9.724243e+06,31.993507,54.363023,1.826479,3.542715
2019-01-25,0.038466,0.967787,1.018436e+07,32.317490,58.515370,1.827445,2.893219
2019-01-28,0.044452,1.377883,1.067242e+07,32.806901,62.894799,1.848342,3.562397
2019-01-29,-0.007686,1.593484,1.078932e+07,33.194319,61.660330,1.788460,3.808422
...,...,...,...,...,...,...,...
2023-11-24,0.000000,0.274359,3.058833e+07,12.087330,55.387377,0.393913,0.330001
2023-11-27,-0.010617,0.247765,3.017891e+07,12.099686,52.589478,0.382205,0.520000
2023-11-28,-0.000821,0.224916,3.061784e+07,12.109061,52.370320,0.377048,0.420000


In [116]:
# Exibe, para o ticker selecionado aleatoriamente, o conjunto de séries temporais das features que será utilizado para teste.
train_test_data[random_ticker]['X_test']

,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14
Date,,,,,,,
2023-12-01,0.046374,0.264506,3.101082e+07,12.279718,66.708115,0.396772,1.220000
2023-12-04,0.025030,0.349257,3.319716e+07,12.422422,70.519607,0.408432,1.570001
2023-12-05,-0.016617,0.395238,3.352663e+07,12.516766,65.163433,0.403544,0.880000
2023-12-06,0.026307,0.478068,3.451209e+07,12.645197,69.174695,0.418290,1.059999
2023-12-07,0.031401,0.588315,3.558389e+07,12.813837,73.249962,0.436270,1.720000
2023-12-08,-0.010842,0.644338,3.590361e+07,12.939992,69.784100,0.424393,1.470000
2023-12-11,-0.002911,0.681216,3.622539e+07,13.043993,68.848580,0.406937,1.320001
2023-12-12,0.023056,0.712908,3.692546e+07,13.176794,72.073958,0.410727,1.910000
2023-12-13,0.003555,0.738116,3.679037e+07,13.298555,72.552167,0.417818,1.780000


In [117]:
# Exibe, para o ticker selecionado aleatoriamente, a série temporal do target que será utilizada para treino.
train_test_data[random_ticker]['y_train']

Date
2019-01-23    31.146366
2019-01-24    33.124378
2019-01-25    34.423378
2019-01-28    35.988075
2019-01-29    35.712532
                ...    
2023-11-24    12.310000
2023-11-27    12.180000
2023-11-28    12.170000
2023-11-29    12.230000
2023-11-30    12.430000
Name: Adj Close, Length: 1224, dtype: float64

In [118]:
# Exibe, para o ticker selecionado aleatoriamente, a série temporal do target que será utilizada para teste.
train_test_data[random_ticker]['y_test']

Date
2023-12-01    13.02
2023-12-04    13.35
2023-12-05    13.13
2023-12-06    13.48
2023-12-07    13.91
2023-12-08    13.76
2023-12-11    13.72
2023-12-12    14.04
2023-12-13    14.09
2023-12-14    14.59
2023-12-15    14.49
2023-12-18    14.24
2023-12-19    14.30
2023-12-20    13.98
2023-12-21    14.35
2023-12-22    14.31
2023-12-26    14.11
2023-12-27    13.99
2023-12-28    13.98
2023-12-29    13.74
Name: Adj Close, dtype: float64

#### *Normalizando as features*

In [119]:
def scale_dfs(train_test_data: dict) -> dict:
    '''
    Description:
        Esta função normaliza os conjuntos de dados de treino e teste das features e do target para cada ticker presente no dicionário 
        "train_test_data" usando o MinMaxScaler. Tais dados serão escalados de modo que todos eles estejam entre 0 e 1.
    Args:
        train_test_data (dict): Um dicionário contendo os dados de treino e teste para cada ticker. Cada chave é um ticker, e os valores 
        são dicionários com as chaves 'X_train', 'X_test', 'y_train', 'y_test', entre outros.
    Return:
        train_test_data (dict): O mesmo dicionário de entrada, mas com os dados de treino e teste normalizados adicionados nas chaves 
        'X_train_scaled', 'X_test_scaled', 'y_train_scaled', 'y_test_scaled' de cada ticker.
    Errors:
        TypeError: É esperado que o parâmetro "train_test_data" seja um dicionário.
    '''
    
    # Verifica se 'train_test_data' é um dicionário.
    if not isinstance(train_test_data, dict):
        raise TypeError("É esperado que o parâmetro 'train_test_data' seja um dicionário.")
    
    # Obtém a lista de tickers.
    tickers = train_test_data.keys()
    
    # Itera sobre cada ticker no dicionário.
    for ticker in tickers:

        # Cria uma instancia do MinMaxScaler para as features do ticker em questão.
        scaler_features = MinMaxScaler()

        # Ajusta a instância criada acima ao conjunto de treino das features do ticker em questão.
        train_test_data[ticker]['X_train_scaled'] = scaler_features.fit_transform(train_test_data[ticker]['X_train'])  

        # Redimensiona y_train para que ele seja bidimensional (Necessário para o MinMaxScaler).
        resized_y_train = train_test_data[ticker]['y_train'].values.reshape(-1, 1)  

        # Cria uma instancia do MinMaxScaler para o target do ticker em questão.
        scaler_target = MinMaxScaler()

        # Ajusta a instância criada acima ao conjunto de treino do target do ticker em questão.
        train_test_data[ticker]['y_train_scaled'] = scaler_target.fit_transform(resized_y_train)  

        # Normaliza o conjunto de teste das features do ticker em questão usando a instância que foi criada e ajustada aos dados de treino
        # desse mesmo ticker.
        train_test_data[ticker]['X_test_scaled'] = scaler_features.transform(train_test_data[ticker]['X_test'])  

        # Redimensiona y_test para que seja bidimensional (Necessário para o MinMaxScaler).
        resized_y_test = train_test_data[ticker]['y_test'].values.reshape(-1, 1) 

        # Normaliza o conjunto de teste do target  do ticker em questão usando a instância que foi criada e ajustada aos dados de treino desse
        # mesmo ticker.
        train_test_data[ticker]['y_test_scaled'] = scaler_target.transform(resized_y_test)
        
    return train_test_data

In [120]:
# Normaliza, para cada ticker presente no dicionário "train_test_data", os conjuntos de dados de treino e teste das features e do target.
train_test_data = scale_dfs(train_test_data)
# Obtem, de forma aleatória, o ticker que representa um dos DataFrames presentes em "dfs_list".
random_ticker = list(train_test_data.keys())[np.random.randint(0,len(train_test_data.keys()))]
# Exibe qual ticker foi escolhido aleatoriamente.
print(random_ticker)

F


In [121]:
# Exibe, para o ticker obtido de forma aleatória, o resultado da normalização do conjunto de treino das features.
train_test_data[random_ticker]['X_train_scaled']

array([[0.32889316, 0.1162648 , 0.15445932, ..., 0.63314448, 0.15959323,
        0.6333181 ],
       [0.47416909, 0.08766792, 0.1760828 , ..., 0.69275783, 0.1706339 ,
        0.66259553],
       [0.47149734, 0.07842361, 0.1814742 , ..., 0.74544005, 0.1739855 ,
        0.65951378],
       ...,
       [0.39290639, 0.10593788, 0.11061974, ..., 0.46552164, 0.1396482 ,
        0.61858217],
       [0.44580024, 0.1157402 , 0.13127964, ..., 0.53615129, 0.16108405,
        0.65116197],
       [0.29190527, 0.09022513, 0.14262564, ..., 0.43847232, 0.17615844,
        0.65307852]])

In [122]:
# Exibe, para o ticker obtido de forma aleatória, o resultado da normalização do conjunto de teste das features.
train_test_data[random_ticker]['X_test_scaled']

array([[0.47420938, 0.07984257, 0.1514683 , 0.37472395, 0.53369335,
        0.18049555, 0.66841015],
       [0.41740406, 0.06170461, 0.16204732, 0.37743806, 0.56616427,
        0.17831248, 0.68374178],
       [0.35699247, 0.06726182, 0.14804067, 0.37897932, 0.53499415,
        0.16731479, 0.61858217],
       [0.40915114, 0.07616717, 0.1491286 , 0.38104497, 0.56112091,
        0.16538315, 0.62049872],
       [0.41976914, 0.08388049, 0.1465104 , 0.38388945, 0.59821349,
        0.15806915, 0.66266069],
       [0.43532013, 0.10503717, 0.15317432, 0.38789554, 0.64922682,
        0.15679791, 0.67032641],
       [0.40032351, 0.12423622, 0.14929802, 0.39185405, 0.66482648,
        0.14802704, 0.66936818],
       [0.40810613, 0.13482893, 0.14550432, 0.39601465, 0.68840764,
        0.13850254, 0.69236572],
       [0.40531644, 0.14551659, 0.15994913, 0.40026928, 0.70929178,
        0.15311979, 0.69332395],
       [0.5951022 , 0.22760033, 0.20751686, 0.41076884, 0.87021189,
        0.19429464, 0.7

In [123]:
# Exibe, para o ticker obtido de forma aleatória, o resultado da normalização do conjunto de treino do target.
train_test_data[random_ticker]['y_train_scaled']

array([[0.1727139 ],
       [0.18396541],
       [0.19521705],
       ...,
       [0.36992889],
       [0.3817697 ],
       [0.36400851]])

In [124]:
# Exibe, para o ticker obtido de forma aleatória, o resultado da normalização do conjunto de teste do target.
train_test_data[random_ticker]['y_test_scaled']

array([[0.38123143],
       [0.38769003],
       [0.38230787],
       [0.38715187],
       [0.39414863],
       [0.4043748 ],
       [0.40760404],
       [0.41244805],
       [0.41675378],
       [0.46196407],
       [0.45873478],
       [0.45389077],
       [0.45873478],
       [0.44689396],
       [0.4759577 ],
       [0.47649592],
       [0.48187803],
       [0.47864879],
       [0.4759577 ],
       [0.4678844 ]])

#### *Ajustando os dados para o formato aceito por modelos LSTM*

In [125]:
def create_time_sequences(train_test_data: dict, sequence_length: int) -> dict:
    '''
        Description:
            Esta função cria e retorna sequências temporais de tamanho fixo para treinamento de redes neurais LSTM. Cada sequência temporal 
            é composta por 'sequence_length' dias consecutivos de dados.
        Args:
            train_test_data (dict): Um dicionário contendo os dados de treino e teste para cada ticker. Cada chave é um ticker, e os valores 
            são dicionários com as chaves 'X_train', 'X_train_scaled', 'X_train_scaled_sequences','X_test', 'X_test_scaled',
            'X_test_scaled_sequence', 'y_train', 'y_train_scaled', 'y_train_scaled_sequence', 'y_test', 'y_test_scaled' e
            'y_test_scaled_sequence'.
            sequence_length (int): O comprimento das sequências temporais de dados que serão geradas para cada ticker.
        Return:
            train_test_data (dict): O mesmo dicionário de entrada, mas com as sequências temporais de dados de treino e teste adicionadas nas 
            chaves 'X_train_scaled_sequences', 'X_test_scaled_sequences', 'y_train_scaled_sequences', 'y_test_scaled_sequences' de cada ticker.
        Errors:
            TypeError: É esperado que o parâmetro "train_test_data" seja um dicionário.
            ValueError: É esperado que o parâmetro "sequence_length" seja um número inteiro positivo.
            ValueError: É esperado que o parâmetro "sequence_length" não seja maior que o número de amostras em "X_train_scaled".
            ValueError: É esperado que o parâmetro "sequence_length" não seja maior que o número de amostras em "X_test_scaled".
    '''
    
    # Verifica se 'train_test_data' é um dicionário.
    if not isinstance(train_test_data, dict):
        raise TypeError("É esperado que o parâmetro 'train_test_data' seja um dicionário.")

    # Verifica se 'sequence_length' é um número inteiro positivo.
    if not isinstance(sequence_length, int) or sequence_length <= 0:
        raise ValueError("É esperado que o parâmetro 'sequence_length' seja um número inteiro positivo.")

    # A ideia aqui é que todos os tickers de treino têm o mesmo tamanho, da mesma forma que todos os tickers de teste tem o mesmo tamanho. Logo,
    # podemos pegar um ticker aleatório para determinar o tamanho.
    random_ticker = list(train_test_data.keys())[np.random.randint(0, len(train_test_data.keys()))]
    
    # Verifica se o sequence_length é maior que o número de amostras disponíveis
    if sequence_length > len(train_test_data[random_ticker]['X_train_scaled']):
        raise ValueError("O 'sequence_length' não pode ser maior que o número de amostras em 'X_train_scaled'.")
    if sequence_length > len(train_test_data[random_ticker]['X_test_scaled']):
        raise ValueError("O 'sequence_length' não pode ser maior que o número de amostras em 'X_test_scaled'.")

    # Calcula o comprimento dos intervalos de treino e teste, assumindo que 'X_train' e 'y_train' têm o mesmo comprimento, tal como 'X_test' e
    # 'y_test'.
    train_interval_length = len(train_test_data[random_ticker]['X_train_scaled']) - sequence_length
    test_interval_length = len(train_test_data[random_ticker]['X_test_scaled']) - sequence_length
    
    # Itera sobre cada ticker no dicionário.
    for ticker in train_test_data.keys():
        # Inicializa listas para armazenar sequências temporais de treino e teste.
        X_train_sequences = []
        X_test_sequences = []
        y_train_sequences = []
        y_test_sequences = []
    
        # Cria sequências para os dados de treino
        for i in range(train_interval_length):
            # Cria uma sequência temporal de 'sequence_length' dias para as features de treino.
            X_train_sequence = train_test_data[ticker]['X_train_scaled'][i: (i + sequence_length)]
            # O alvo será o valor no dia seguinte após a sequência temporal.
            y_train_sequence = train_test_data[ticker]['y_train_scaled'][i + sequence_length]
            # Adiciona as sequências temporais às listas correspondentes.
            X_train_sequences.append(X_train_sequence)
            y_train_sequences.append(y_train_sequence)

        # Cria sequências para os dados de teste
        for j in range(test_interval_length):
            # Cria uma sequência temporal de 'sequence_length' dias para as features de teste.
            X_test_sequence = train_test_data[ticker]['X_test_scaled'][j: (j + sequence_length)]
            # O alvo será o valor no dia seguinte após a sequência temporal.
            y_test_sequence = train_test_data[ticker]['y_test_scaled'][j + sequence_length]
            # Adiciona as sequências temporais às listas correspondentes.
            X_test_sequences.append(X_test_sequence)
            y_test_sequences.append(y_test_sequence)
            
        # Adiciona as listas de sequências temporais ao dicionário do ticker correspondente.
        train_test_data[ticker]['X_train_scaled_sequences'] = np.array([np.array(arr) for arr in X_train_sequences])
        train_test_data[ticker]['X_test_scaled_sequences'] = np.array([np.array(arr) for arr in X_test_sequences])
        train_test_data[ticker]['y_train_scaled_sequences'] = np.array(y_train_sequences)
        train_test_data[ticker]['y_test_scaled_sequences'] = np.array(y_test_sequences)
        
    # Retorna o dicionário atualizado com os novos valores das chaves 'X_train_scaled_sequences', 'X_test_scaled_sequences'
    # 'y_train_scaled_sequences' e 'y_test_scaled_sequences'.
    return train_test_data


In [126]:
# Cria sequências temporais (de treino e teste) de tamanho "setup['sequence_length']" no intuito de usa-lás na rede neural que iremos criar.
train_test_data = create_time_sequences(train_test_data, setup['sequence_length'])
# Obtem, de forma aleatória, o ticker que representa um dos DataFrames presentes em "dfs_list".
random_ticker = list(train_test_data.keys())[np.random.randint(0,len(train_test_data.keys()))]
# Exibe qual ticker foi escolhido aleatoriamente.
print(random_ticker)

AAL


In [127]:
# Exibe a quantidade de sequências temporais de features que serão utilizadas no treinamento da rede neural.
len(train_test_data[random_ticker]['X_train_scaled_sequences'])

1217

In [128]:
# Exibe algumas das sequências temporais de features que serão utilizadas no treinamento da rede neural.
train_test_data[random_ticker]['X_train_scaled_sequences']

array([[[0.40447043, 0.16943409, 0.03005749, ..., 0.45187391,
         0.67691892, 0.54628015],
        [0.55476576, 0.12587564, 0.02949554, ..., 0.56540249,
         0.70923703, 0.72199578],
        [0.5183768 , 0.1681344 , 0.03229972, ..., 0.62805907,
         0.70970154, 0.69581698],
        ...,
        [0.44569124, 0.30512324, 0.03598659, ..., 0.67551474,
         0.69095112, 0.73270546],
        [0.4599642 , 0.3445453 , 0.03653343, ..., 0.67769008,
         0.64702982, 0.69502375],
        [0.43289735, 0.35070734, 0.03130106, ..., 0.6365361 ,
         0.62307958, 0.72715208]],

       [[0.55476576, 0.12587564, 0.02949554, ..., 0.56540249,
         0.70923703, 0.72199578],
        [0.5183768 , 0.1681344 , 0.03229972, ..., 0.62805907,
         0.70970154, 0.69581698],
        [0.52780307, 0.25792004, 0.03527415, ..., 0.69414218,
         0.71975219, 0.72278909],
        ...,
        [0.4599642 , 0.3445453 , 0.03653343, ..., 0.67769008,
         0.64702982, 0.69502375],
        [0.4

In [129]:
# Exibe a quantidade de sequências temporais de features que serão utilizadas no teste da rede neural.
len(train_test_data[random_ticker]['X_test_scaled_sequences']) 

13

In [130]:
# Exibe algumas das sequências temporais de features que serão utilizadas no teste da rede neural.
train_test_data[random_ticker]['X_test_scaled_sequences']

array([[[0.53083008, 0.01415972, 0.15922519, 0.09605425, 0.75168297,
         0.02159297, 0.62837567],
        [0.49721534, 0.03271493, 0.17254967, 0.10165955, 0.80919624,
         0.02720062, 0.64248291],
        [0.43162619, 0.04278201, 0.17455761, 0.10536528, 0.72837459,
         0.02484966, 0.61467152],
        [0.49922749, 0.06091666, 0.18056346, 0.11040994, 0.78890227,
         0.03194244, 0.62192663],
        [0.50724933, 0.08505372, 0.18709543, 0.11703399, 0.85039576,
         0.0405899 , 0.64852884],
        [0.44072058, 0.0973194 , 0.18904395, 0.12198924, 0.79809785,
         0.03487771, 0.63845226],
        [0.45321104, 0.10539334, 0.19100501, 0.12607431, 0.78398138,
         0.02648161, 0.63240632]],

       [[0.49721534, 0.03271493, 0.17254967, 0.10165955, 0.80919624,
         0.02720062, 0.64248291],
        [0.43162619, 0.04278201, 0.17455761, 0.10536528, 0.72837459,
         0.02484966, 0.61467152],
        [0.49922749, 0.06091666, 0.18056346, 0.11040994, 0.78890227,
  

In [131]:
# Exibe a quantidade de valores alvo que serão utilizados no treinamento da rede neural.
len(train_test_data[random_ticker]['y_train_scaled_sequences'])

1217

In [132]:
# Exibe alguns dos valores alvos que serão utilizados no treinamento da rede neural.
train_test_data[random_ticker]['y_train_scaled_sequences']

array([[0.96693498],
       [0.99099723],
       [1.        ],
       ...,
       [0.11422764],
       [0.11641729],
       [0.1237162 ]])

In [133]:
# Exibe a quantidade de valores alvo que serão utilizados no teste da rede neural.
len(train_test_data[random_ticker]['y_test_scaled_sequences'])

13

In [134]:
# Exibe alguns dos valores alvos que serão utilizados no teste da rede neural.
train_test_data[random_ticker]['y_test_scaled_sequences']

array([[0.18247226],
       [0.18429699],
       [0.20254422],
       [0.19889476],
       [0.18977114],
       [0.19196083],
       [0.18028258],
       [0.19378555],
       [0.19232578],
       [0.18502686],
       [0.18064753],
       [0.18028258],
       [0.17152392]])

# **Criando a rede neural**

In [135]:
"""
    Este código define, constrói e compila um modelo LSTM (Long Short-Term Memory) para previsões de séries temporais.

    Parâmetros dos métodos utilizados:
    1. `LSTM`:
        - `units`: Número de neurônios na camada LSTM.
        - `return_sequences`: Se `True`, retorna a sequência completa de saídas para cada unidade LSTM; se `False`, retorna apenas 
           a saída final.
        - `input_shape`: Tupla que define a forma da entrada (tamanho da sequência de tempo, número de features).

    2. `Dropout`:
        - `rate`: Fração de neurônios a serem descartados durante o treinamento para evitar overfitting.

    3. `Dense`:
        - `units`: Número de neurônios na camada densa (totalmente conectada). No contexto de regressão, geralmente é 1.

    4. `compile`:
        - `optimizer`: Algoritmo de otimização utilizado para ajustar os pesos do modelo. 'adam' é uma escolha comum para LSTM.
        - `loss`: Função de perda que o modelo tentará minimizar durante o treinamento. Neste caso, 'mean_squared_error' (erro quadrático médio) é usado para problemas de regressão.
"""

# Como os DataFrames de todos os tickers possuem o mesmo número de features, podemos escolher um ticker aleatório para obter o número de
# features de todos os Dataframes.
random_ticker = list(train_test_data.keys())[np.random.randint(0, len(train_test_data.keys()))]
    
# Define o número de features
n_features = train_test_data[random_ticker]['X_test'].shape[1]

# Definindo o modelo LSTM
# Utilizamos o modelo 'Sequential', que permite empilhar camadas de forma linear. Esse tipo de modelo é adequado para a maioria das 
# arquiteturas de rede neural onde as camadas são adicionadas uma após a outra.
model = Sequential()

# Adiciona a primeira camada LSTM com 50 unidades (neurônios).
model.add(LSTM(units=50, return_sequences=True, input_shape=(setup['sequence_length'], n_features)))

# Adiciona uma camada de Dropout. Tal camada é usada para evitar overfitting durante o treinamento do modelo. Ao definir `rate=0.2`, 
# estamos especificando que 20% dos neurônios da camada anterior serão desligados aleatoriamente em cada atualização do ciclo de treinamento.
model.add(Dropout(0.2))

# Adiciona uma segunda LSTM. Ao definirmos`return_sequences=False` indicamos que esta é a última camada LSTM na rede, e 
# ela só retorna a última saída em vez de toda a sequência.
model.add(LSTM(units=50, return_sequences=False))

# Adiciona uma segunda camada de Dropout. Assim como antes, 20% dos neurônios serão desativados em cada iteração de treinamento.
model.add(Dropout(0.2))

# Adiciona a camada de saída. Tal camada é uma camada densa totalmente conectada com um único neurônio (units=1). Isso é adequado para 
# problemas de regressão onde a previsão final é um único valor contínuo.
model.add(Dense(units=1))

# Compila o modelo criado. O modelo é compilado com o otimizador 'adam', que é eficiente para grandes volumes de dados e adequado para
# problemas de regressão. Além disso, setamos a função de perda como sendo a 'mean_squared_error' (MSE), que é uma escolha comum para medir 
# o erro médio ao quadrado entre as previsões e os valores reais.
model.compile(optimizer='adam', loss='mean_squared_error')

# A variável 'model' agora contém o modelo LSTM compilado e pode ser usada para treinamento e previsões.

/home/user/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [136]:
def train_model_and_get_results(model: Sequential, train_test_data: dict) -> dict:
   '''
      Description:
         Esta função treina um modelo LSTM para cada ticker presente no dicionário `train_test_data`. O modelo é treinado usando os dados de
         treinamento normalizados e as sequências temporais geradas previamente. Após o treinamento, o modelo realiza previsões com os dados 
         de teste disponíveis e avalia o desempenho usando a métrica Root Mean Squared Error (RMSE). As previsões e os valores de RMSE para
         cada ticker são armazenados no dicionário `train_test_data`.
      
      Args:
         model (Sequential): O modelo LSTM a ser treinado. Deve ser uma instância da classe Sequential da biblioteca keras.
         train_test_data (dict): Um dicionário contendo os dados de treino e teste para cada ticker. Cada chave é um ticker, e os valores 
         são dicionários com as chaves 'X_train_scaled_sequences', 'X_test_scaled_sequence', 'y_train_scaled_sequences', 'y_test_scaled_sequence',
         'predicted', 'RMSE', entre outras.

      Return:
         train_test_data (dict): O mesmo dicionário de entrada, mas com as previsões e métricas de avaliação (RMSE) adicionadas nas chaves 
         'predicted' e 'RMSE' de cada ticker.
   '''
   
   # Itera sobre cada ticker presente em "train_test_data.keys()".
   for ticker in train_test_data.keys():
      
      """
         Este código treina o modelo LSTM usando o método `fit` do módulo Keras da biblioteca tensorflow.

         Parâmetros do método `fit`:
         1. `x`: Dados de entrada para treinamento. No contexto de séries temporais com LSTM, este é um array de sequências escaladas.
         2. `y`: Dados de saída (alvo) para treinamento. Correspondem aos valores que o modelo deve prever com base nos dados de entrada.
         3. `epochs`: Número de vezes que o modelo passará por todo o conjunto de dados de treinamento. Um número maior de épocas pode melhorar o 
            ajuste do modelo, mas também aumenta o risco de overfitting.
         4. `batch_size`: Número de amostras que o modelo processa antes de atualizar os pesos. Tamanhos de batch menores podem resultar em um 
            treinamento mais ruidoso, mas permitem uma melhor generalização.
         5. `validation_split`: Proporção dos dados de treinamento que será usada para validação. Ajuda a monitorar a performance do modelo em 
            dados que ele não viu durante o treinamento.
         6. `verbose`: Nível de verbosidade do processo de treinamento. `verbose=1` exibe uma barra de progresso detalhada durante o treinamento.
      """

      # Treina o modelo
      # O método `fit` treina o modelo LSTM usando os dados de entrada (`X_train_scaled_sequence`) e as saídas correspondentes 
      # (`y_train_scaled_sequence`).
      model.fit(
         train_test_data[ticker]['X_train_scaled_sequences'],  # Dados de entrada de treino (sequências temporais)
         train_test_data[ticker]['y_train_scaled_sequences'],  # Dados de saída de treino (valores de previsão para cada sequência)
         epochs=50,  # Número de épocas: o modelo passará 50 vezes por todo o conjunto de dados de treinamento
         batch_size=32,  # Tamanho do batch: o modelo ajusta os pesos após processar cada lote de 32 amostras
         validation_split=0.2,  # 20% dos dados de treinamento serão usados para validação.
         verbose=1  # Nível de verbosidade: 1 mostra uma barra de progresso e resultados após cada época
      )
      
      """
            Este bloco de código faz previsões sobre um conjunto de dados de teste usando o modelo LSTM treinado acima, e, em seguida,
            avalia a precisão dessas previsões usando a métrica de Root Mean Squared Error (RMSE).
      """

      
      # Realiza a previsão de valores para dados não vistos até então.
      train_test_data[ticker]['predicted'] = model.predict(train_test_data[ticker]['X_test_scaled_sequences'])

      # Avalia, utilizando a métrica RMSE, o resultado das previsões feitas acima.
      train_test_data[ticker]['RMSE'] = np.sqrt(mean_squared_error(train_test_data[ticker]['y_test_scaled_sequences'],
                                                                  train_test_data[ticker]['predicted']))
   
   return train_test_data

In [137]:
# Realiza, para cada um dos tickers presentes na variável "train_test_data", o treinamento do modelo LSTM criado e salva os resultados obtidos
# por esse modelo em suas tentativas de previsão.
train_test_data = train_model_and_get_results(model, train_test_data)

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 0.1197 - val_loss: 0.0041
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0159 - val_loss: 9.9603e-04
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0085 - val_loss: 0.0011
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0072 - val_loss: 8.9435e-04
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0063 - val_loss: 8.2165e-04
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0055 - val_loss: 0.0011
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0059 - val_loss: 0.0017
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0054 - val_loss: 7.0298e-04
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0059 - val_loss: 6.8150e-04
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0056 - val_loss: 6.5233e-04
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0041 - val_loss: 6.7799e-04
Epoch 12/50
31/31 ━━━━━━━━━━━━━━━━━

# **Resultados obtidos**

In [138]:
def plot_multiples_time_series_line_graphs(data_list: list, serie_title: Optional[str] = "", serie_xaxis_title: Optional[str] = "", 
                               serie_yaxis_title: Optional[str] = "") -> None:
    '''
        Description:
            Esta função exibe em um mesmo plot múltiplos gráficos de séries temporais. 
        Args:
            data_list (list): Lista contendo as séries temporais cujos gráficos serão exibidos no plot. Os valores de tais séries temporais
                              estarão no eixo y, enquanto os índices dessas séries estarão no eixo x.
            title (string) [Optional]: Título do plot.
            xaxis_title (string) [Optional]: Título do eixo x do plot.
            yaxis_title (string) [Optional]: Título do eixo y do plot.
        Return:
            None: A função exibe os gráficos, mas não retorna nenhum valor.  
        Errors:
            TypeError: É esperado que todas os elementos da lista "data_list" sejam objetos do tipo pd.Series, isto é, que sejam séries temporais.
            ValueError: É esperado que todas as séries temporais presentes na variável "data_list" possuam os mesmos índices. 
            TypeError: É esperado que todas as séries temporais presentes na variável data_list possuam um atributo "name".
    '''    

    # Verifica se todos os elementos presentes na lista "data_list" são séries temporais.
    are_all_data_time_series = all(isinstance(df, pd.Series) for df in data_list)
    
    # Retorna um erro caso algum dos dados presentes na variável "data_list" não seja uma série temporal
    if not are_all_data_time_series:
        raise TypeError("Todos os dados presentes no parâmetro 'data_list' devem ser séries temporais.")
    
    # Verifica se as séries temporais presentes na variável "data_list" possuem os mesmos índices.
    are_all_index_equal = all(df.index.equals(data_list[0].index) for df in data_list)
    
    # Retorna um erro caso as séries temporais possuam índices diferentes.
    if not are_all_index_equal:
        raise ValueError("Todos as séries temporais devem possuir os mesmos índices.")
    
    # Verifica se todas as séries temporais presentes na variável "data_list" possuem o atributo "name".
    all_time_series_have_names = all(hasattr(df,"name") for df in data_list)
    
    # Retorna um erro caso uma das séries temporais presentes na variável "data_list" não possua o atributo "name".
    if not all_time_series_have_names:
        raise TypeError("Todas as séries temporais devem possuir o atributo 'name'")
    
    # Cria uma lista de timestamps que representará o eixo x do gráfico que será plotado.
    x_axis = data_list[0].index.tolist() # Observe que só podemos fazer isso pois temos certeza que todas as séries temporais possuem os mesmos índices.
    # Cria a figura onde será plotado o gráfico.
    fig = go.Figure()

    # Adiciona cada série temporal ao gráfico.
    for time_serie in data_list:
        # Plota o gráfico (Data x Valor da Ação) do ticker em questão.
        fig.add_trace(go.Scatter(x=x_axis, y=time_serie.values, mode="lines", name=time_serie.name))
    
    # Atualiza o layout para permitir destaque ao clicar na legenda.
    fig.update_layout(
        # Seta um título para o plot.
        title=serie_title,
        # Seta um título para o eixo x do plot.
        xaxis_title=serie_xaxis_title,
        # Seta um título para o eixo y do plot.
        yaxis_title=serie_yaxis_title,   
    )
    
    # Exibe o gráfico criado.
    fig.show()

In [139]:
for ticker in train_test_data.keys():
        
        # Redimensiona y_train para que ele seja bidimensional (Necessário para o MinMaxScaler).
        resized_y_train = train_test_data[ticker]['y_train'].values.reshape(-1, 1)  

        # Cria uma instancia do MinMaxScaler para o target do ticker em questão.
        scaler_target = MinMaxScaler()

        # Ajusta a instância criada acima ao conjunto de treino do target do ticker em questão.
        scaler_target.fit_transform(resized_y_train)
        
        # Volta os valores de "y_test_scaled_sequences" para a escala normal e associa a esses valores as suas datas originais, criando assim
        # uma série temporal.
        real_y = pd.Series(scaler_target.inverse_transform(train_test_data[ticker]['y_test_scaled_sequences']).flatten(),
                      index= train_test_data[ticker]['y_test'].index[len(train_test_data[ticker]['y_test']) - len(train_test_data[ticker]['y_test_scaled_sequences']):]
        )
        # Adiciona um nome a série temporal "real_y"
        real_y.name = f"Preço real de {ticker}"
        
        # Volta os valores estimados para a escala normal e associa a esses valores as suas datas originais, criando assim
        # uma série temporal.
        predicted_y = pd.Series(
            scaler_target.inverse_transform(train_test_data[ticker]['predicted']).flatten(),
            index=train_test_data[ticker]['y_test'].index[len(train_test_data[ticker]['y_test']) - len(train_test_data[ticker]['y_test_scaled_sequences']):]
        )
        # Adiciona um nome a série temporal "predicted_y"
        predicted_y.name = f"Preço estimado de {ticker}"
        
        # Plota um gráfico de linha comparativo para as variáveis "real_y" e "predicted_y" em questão.
        plot_multiples_time_series_line_graphs([real_y, predicted_y], f"Preço real de {ticker} x Preço estimado de {ticker}", "Data", "Preço")   

In [140]:
# Exibe o Erro Quadrático Médio associado as previsões feitas para cada um dos tickers presentes em "train_test_data.keys()".
for i, ticker in enumerate(train_test_data.keys()):
    print(f"O ticker {ticker} teve um Erro Quadrático Médio de aproximadamente {train_test_data[ticker]['RMSE']*100:.2f}% associado as previsões feitas para ele.")

O ticker AMCR teve um Erro Quadrático Médio de aproximadamente 1.25% associado as previsões feitas para ele.
O ticker AAL teve um Erro Quadrático Médio de aproximadamente 3.09% associado as previsões feitas para ele.
O ticker T teve um Erro Quadrático Médio de aproximadamente 2.49% associado as previsões feitas para ele.
O ticker F teve um Erro Quadrático Médio de aproximadamente 1.74% associado as previsões feitas para ele.
O ticker NVDA teve um Erro Quadrático Médio de aproximadamente 10.36% associado as previsões feitas para ele.


# **Alguns pontos interessantes**

- Embora uma das consequências do Teorema da Aproximação Universal nos garanta que qualquer função contínua possa ser aproximada por uma rede neural de camada simples e com número finito de neurônios, tal teorema apenas nos diz que essa rede neural existe, mas não nos fornece qualquer informação sobre ela. Na prática, *nos dias atuais*, é praticamente impossível de se criar um modelo de IA que reflita os preços reais com ligeira perfeição. Mas, os modelos de IA atuais podem, se bem criados, serem muito bons em revelarem tendências futuras

- Quais features precisariamos adicionar/remover do modelo apresentado para que ele refletisse de forma melhor os preços reais ?

###